In [11]:
# Forward Chaining Reasoning in Python

# Facts and rules from the input
facts = {
    "American(Robert)": True,
    "Missile(T1)": True,
    "Owns(A, T1)": True,
    "Enemy(A, America)": True,
}

rules = [
    # Rule 1: Missiles are weapons
    {"if": ["Missile(x)"], "then": ["Weapon(x)"]},

    # Rule 2: Selling weapons to a hostile country makes someone a criminal
    {"if": ["American(p)", "Weapon(q)", "Sells(p, q, r)", "Hostile(r)"], "then": ["Criminal(p)"]},

    # Rule 3: All missiles were sold to country A by Robert
    {"if": ["Missile(x)", "Owns(A, x)"], "then": ["Sells(Robert, x, A)"]},

    # Rule 4: The enemy of America is hostile
    {"if": ["Enemy(x, America)"], "then": ["Hostile(x)"]},
]

# Query to prove
query = "Criminal(Robert)"

# Helper function to unify a fact with a condition
def unify(condition, fact):
    predicate_c, args_c = condition.split("(")
    predicate_f, args_f = fact.split("(")

    if predicate_c != predicate_f:
        return None  # Predicates must match

    args_c = args_c[:-1].split(",")
    args_f = args_f[:-1].split(",")

    if len(args_c) != len(args_f):
        return None

    substitutions = {}

    for var, const in zip(args_c, args_f):
        if var.islower():  # Variable
            substitutions[var] = const
        elif var != const:  # Constant mismatch
            return None

    return substitutions

# Apply substitutions to a statement
def apply_substitutions(statement, substitutions):
    predicate, args = statement.split("(")
    args = args[:-1].split(",")
    new_args = [substitutions.get(arg, arg) for arg in args]
    return f"{predicate}({', '.join(new_args)})"

# Forward chaining algorithm
def forward_chaining(facts, rules, query):
    inferred = set(facts.keys())  # Start with known facts

    while True:
        new_inferences = set()

        for rule in rules:
            # Check if the "if" part of the rule is satisfied
            conditions = rule["if"]
            substitutions_list = [{}]

            for condition in conditions:
                temp_substitutions = []

                for fact in inferred:
                    subs = unify(condition, fact)
                    if subs is not None:
                        for existing_subs in substitutions_list:
                            combined_subs = existing_subs.copy()
                            combined_subs.update(subs)
                            temp_substitutions.append(combined_subs)

                if not temp_substitutions:
                    break
                substitutions_list = temp_substitutions

            # If all conditions are satisfied, infer the "then" part
            if substitutions_list:
                for subs in substitutions_list:
                    for conclusion in rule["then"]:
                        new_fact = apply_substitutions(conclusion, subs)
                        new_inferences.add(new_fact)

        # Add new inferences to facts
        if query in new_inferences:
            return True  # Query is proven

        if not new_inferences - inferred:
            return False  # No new facts can be inferred

        inferred.update(new_inferences)

# Run the algorithm and print the result
if forward_chaining(facts, rules, query):
    print(f"The query '{query}' is proven.")
else:
    print(f"The query '{query}' cannot be proven.")


The query 'Criminal(Robert)' is proven.
